## Prep

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

In [2]:
fb_data = pd.read_excel('Daily Show Advertising data.xlsx', sheet_name='All data')

fb_data[['Ad_type', 'Ad_content', 'na', 'na1']] = fb_data['Ad name'].str.split('|', expand=True)
fb_data = fb_data.drop(['na', 'na1'], axis=1)

# fb_data['Starts'] = pd.to_datetime(fb_data['Starts'])
# fb_data['Ends'] = pd.to_datetime(fb_data['Ends'])
fb_data['Day'] = pd.to_datetime(fb_data['Day'])

ad_data = fb_data.drop(['Ad name', 'Attribution setting', 'Starts', 'Ends', 'Reporting starts', 'Reporting ends'], axis=1)
ad_data = ad_data[['Ad_type', 'Ad_content', 'Day', 'Amount spent (USD)']]
ad_data.columns = ['type', 'performance', 'date', 'ad spent']

ad_data = ad_data[~ad_data['performance'].isna()]
ad_data = ad_data.groupby('date')['ad spent'].sum().reset_index()

ad_data['ad spent^2'] = ad_data['ad spent'] ** 2

# ad_data['original date'] = ad_data['date']
ad_data['date'] = ad_data['date'] - timedelta(days=30)

ad_data

,date,ad spent,ad spent^2
0,2020-11-12,16.36,267.6496
1,2020-11-13,8.39,70.3921
2,2020-11-14,12.16,147.8656
3,2020-11-15,17.93,321.4849
4,2020-11-16,11.68,136.4224
...,...,...,...
847,2023-10-12,239.98,57590.4004
848,2023-10-13,249.79,62395.0441
849,2023-10-14,244.69,59873.1961
850,2023-10-15,238.46,56863.1716


In [3]:
ticket_sales = pd.read_excel('SSO Report Jan 2020 thru Oct 2023.xlsx')
memb_hist = pd.read_excel("Member History (1).xlsx", sheet_name='MemHistory')
sf_zipcodes = pd.read_excel('San_Francisco_ZIP_Codes.xlsx', sheet_name='zip_codes')

In [4]:
# streaming members
subs = memb_hist[memb_hist['Membership Level'].isin(
    ['Annual Digital', 'Corporate Annual Digital', 'Corporate One Month Digital', 'Monthly Digital', 'One Month Digital', 'Three Month Digital Membership'])]
subs = subs.sort_values(
    ['Constituent ID', 'Initiation Date'], ascending=[True, False])

# memb_lvl
# subs = subs[['Constituent ID', 'Membership Level', 'Initiation Date', 'Expiration Date']]
# subs['memb_lvl'] = subs['Membership Level'].apply(lambda x: True if 'Month' in x else False)
mem_cols = pd.get_dummies(
    subs['Membership Level'], drop_first=True, prefix='membership').columns
subs = pd.get_dummies(
    subs, columns=['Membership Level'], drop_first=True, prefix='membership')

# join
subs = pd.merge(ticket_sales, subs, how='left',
                left_on='customer_no', right_on='Constituent ID')
subs[mem_cols] = subs[mem_cols].fillna(False)

# # customers living in SF


def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return 99999


subs['zip'] = subs['Primary Address Postcode'].apply(lambda x: str(x)[:5])
subs['zip'] = subs['zip'].apply(lambda x: '99999' if len(x) < 5 else x)
subs['zip'] = subs['zip'].fillna('99999')
subs['zip'] = subs['zip'].apply(convert_to_int)
subs['% customers living in SF / total'] = subs['zip'].isin(sf_zipcodes['Zip Code'])
# subs['# purchases'] = subs['zip'].isin(sf_zipcodes['Zip Code'])

# drop cols
drop_columns = ['paid_flag', 'printed_flag', 'seated_flag', 'created_by', 'last_update_by', 'last_updated_dt', 'location', 'account_no',
                'card_expiry_dt', 'act_name', 'description', 'perf_dt', 'mos_desc', 'ord_notes', 'memb_level', 'tot_bal_amt', 'delivery_desc', 'channel']
subs = subs.drop(drop_columns, axis=1).reset_index(drop=True)

# sub
subs['create_dt'] = pd.to_datetime(
    subs['create_dt'], format='%m/%d/%Y %H:%M:%S:%f').dt.date
# subs['sub'] = (subs['create_dt'] >= subs['Initiation Date']) & (
#     subs['create_dt'] <= subs['Expiration Date'])
subs['date'] = subs['create_dt']

# mos
subs = subs[subs['mos'] != 11]
subs['% online purchases / total'] = subs['mos'].apply(
    lambda x: True if x in (3, 10, 46) else False)

# sales
subs['live shows ticket sales'] = subs['tot_due_amt']
subs = subs[subs['live shows ticket sales'] > 0]

# num_seats
subs['% multiple seats purchase / total'] = subs['num_seats'].apply(
    lambda x: True if x > 1 else False)

# subs = subs[['create_dt', 'customer_no', 'order_no', 'perf_code', 'live shows ticket sales', 'memb_lvl', 'sub', '% online purchases / total', '% multiple seats purchase / total', '# customers living in SF']]
subs

,customer_no,order_no,perf_code,num_seats,create_dt,mos,tot_due_amt,Constituent ID,Membership Organization,Inception Date,...,membership_Corporate One Month Digital,membership_Monthly Digital,membership_One Month Digital,membership_Three Month Digital Membership,zip,% customers living in SF / total,date,% online purchases / total,live shows ticket sales,% multiple seats purchase / total
0,275611,1770305,LSLAVAY,1,2020-12-20,10,20.0,275611.0,Digital Membership,2020-07-03,...,False,False,False,False,99999,False,2020-12-20,True,20.0,False
1,177677,2363644,LILA1106,2,2022-10-15,3,99.0,NaN,NaN,NaT,...,False,False,False,False,99999,False,2022-10-15,True,99.0,True
6,377405,2391742,KOALA1127L,2,2022-11-22,3,109.0,NaN,NaN,NaT,...,False,False,False,False,99999,False,2022-11-22,True,109.0,True
7,317409,2482020,RAZAV0319E,1,2023-03-07,3,29.0,NaN,NaN,NaT,...,False,False,False,False,99999,False,2023-03-07,True,29.0,False
8,386220,2539386,DELVON0607,2,2023-06-01,10,52.5,NaN,NaN,NaT,...,False,False,False,False,99999,False,2023-06-01,True,52.5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178263,336475,2532053,MILES0525,2,2023-05-23,14,210.0,336475.0,Digital Membership,2020-04-03,...,False,False,True,False,94122,True,2023-05-23,False,210.0,True
178264,336475,2599650,CLARKE0810,1,2023-08-08,10,37.5,336475.0,Digital Membership,2020-04-03,...,False,False,True,False,94122,True,2023-08-08,True,37.5,False
178265,336475,2539918,COLL0907E,1,2023-06-01,10,35.0,336475.0,Digital Membership,2020-04-03,...,False,False,True,False,94122,True,2023-06-01,True,35.0,False
178266,336475,2623307,MCBRID0924,1,2023-08-29,10,55.0,336475.0,Digital Membership,2020-04-03,...,False,False,True,False,94122,True,2023-08-29,True,55.0,False


In [5]:
subs = subs[['date', 'live shows ticket sales', '% online purchases / total',
             '% multiple seats purchase / total', '% customers living in SF / total'] + mem_cols.to_list()]

sales = subs.groupby('date').agg({
    'live shows ticket sales': 'sum',
    'membership_Corporate Annual Digital': 'sum',
    'membership_Corporate One Month Digital': 'sum',
    'membership_Monthly Digital': 'sum',
    'membership_One Month Digital': 'sum',
    'membership_Three Month Digital Membership': 'sum',
    '% online purchases / total': lambda x: x.sum() / len(x) * 100,
    '% multiple seats purchase / total': lambda x: x.sum() / len(x) * 100,
    '% customers living in SF / total': lambda x: x.sum() / len(x) * 100,
    # '# purchases': 'count'
}).reset_index()

sales['date'] = pd.to_datetime(sales['date'])

sales

,date,live shows ticket sales,membership_Corporate Annual Digital,membership_Corporate One Month Digital,membership_Monthly Digital,membership_One Month Digital,membership_Three Month Digital Membership,% online purchases / total,% multiple seats purchase / total,% customers living in SF / total
0,2020-01-01,14913.69,0,0,7,6,0,98.837209,88.372093,1.162791
1,2020-01-02,31776.00,0,0,69,16,2,93.121693,93.121693,19.576720
2,2020-01-03,18251.30,0,0,5,9,0,77.419355,79.032258,6.451613
3,2020-01-04,14743.75,0,0,2,2,0,88.505747,80.459770,2.298851
4,2020-01-05,18110.50,0,0,5,1,0,96.875000,78.125000,9.375000
...,...,...,...,...,...,...,...,...,...,...
1253,2023-10-15,5781.25,0,0,8,2,0,57.142857,58.928571,0.000000
1254,2023-10-16,1276.00,0,0,0,0,0,100.000000,80.000000,0.000000
1255,2023-10-17,1763.00,0,0,13,0,0,40.000000,72.000000,52.000000
1256,2023-10-18,83164.50,0,0,0,4,0,55.882353,67.647059,14.705882


In [6]:
df = pd.merge(ad_data, sales, on='date')

# quarter
quarters_dummies = pd.get_dummies(df['date'].dt.quarter, prefix='quarter', drop_first=True, dtype=int)
df = pd.concat([df, quarters_dummies], axis=1)

# weekday or weekend
df['weekday_or_weekend'] = np.where(df['date'].dt.day_of_week < 5, 0, 1)

# monthly membership
df['membership_Monthly Digital'] = df['membership_Monthly Digital'] + df['membership_One Month Digital']
df = df.drop(['membership_One Month Digital'], axis=1)

# renaming columns
cols = [x.replace('_', ' ') for x in df.columns]
df.columns = cols

df.head()

,date,ad spent,ad spent^2,live shows ticket sales,membership Corporate Annual Digital,membership Corporate One Month Digital,membership Monthly Digital,membership Three Month Digital Membership,% online purchases / total,% multiple seats purchase / total,% customers living in SF / total,quarter 2,quarter 3,quarter 4,weekday or weekend
0,2020-11-12,16.36,267.6496,15429.0,0,0,932,9,99.137931,0.156740,10.579937,0,0,1,0
1,2020-11-13,8.39,70.3921,1965.0,0,0,114,1,92.857143,2.857143,2.857143,0,0,1,0
2,2020-11-14,12.16,147.8656,4704.0,0,0,195,0,100.000000,1.277955,4.153355,0,0,1,1
3,2020-11-15,17.93,321.4849,9675.0,0,0,575,4,100.000000,0.487211,6.577345,0,0,1,1
4,2020-11-16,11.68,136.4224,275.0,0,0,3,0,100.000000,4.761905,0.000000,0,0,1,0


In [7]:
# df.to_excel('df.xlsx', index=False)

## Models

In [8]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import VotingRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [9]:
df.describe()

,date,ad spent,ad spent^2,live shows ticket sales,membership Corporate Annual Digital,membership Corporate One Month Digital,membership Monthly Digital,membership Three Month Digital Membership,% online purchases / total,% multiple seats purchase / total,% customers living in SF / total,quarter 2,quarter 3,quarter 4,weekday or weekend
count,846,846.000000,8.460000e+02,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000
mean,2022-02-08 23:34:28.085106432,359.223463,2.723682e+05,17367.539480,0.002364,0.026005,42.303783,0.319149,91.361473,54.679826,7.677936,0.212766,0.319149,0.260047,0.287234
min,2020-11-12 00:00:00,8.390000,7.039210e+01,5.000000,0.000000,0.000000,0.000000,0.000000,55.721393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2021-06-17 06:00:00,109.632500,1.201980e+04,1981.250000,0.000000,0.000000,9.000000,0.000000,85.956072,18.750000,1.219512,0.000000,0.000000,0.000000,0.000000
50%,2022-01-14 12:00:00,250.765000,6.288310e+04,14410.590000,0.000000,0.000000,22.000000,0.000000,92.920821,68.512310,4.804255,0.000000,0.000000,0.000000,0.000000
75%,2022-08-13 18:00:00,451.425000,2.037850e+05,22353.440000,0.000000,0.000000,44.000000,0.000000,100.000000,79.971264,10.960181,0.000000,1.000000,1.000000,1.000000
max,2023-10-16 00:00:00,2425.190000,5.881547e+06,768739.500000,1.000000,2.000000,1126.000000,10.000000,100.000000,100.000000,71.428571,1.000000,1.000000,1.000000,1.000000
std,NaN,378.809116,6.304185e+05,38168.195759,0.048593,0.166509,86.170005,0.910055,9.109505,32.583489,9.021352,0.409506,0.466423,0.438920,0.452739


### Ad spent

In [10]:
X = df.drop(['date', 'live shows ticket sales', 'ad spent^2',
            'membership Corporate Annual Digital', 'membership Corporate One Month Digital', 
            'membership Three Month Digital Membership'], axis=1)

y = df['live shows ticket sales']

df.describe()

,date,ad spent,ad spent^2,live shows ticket sales,membership Corporate Annual Digital,membership Corporate One Month Digital,membership Monthly Digital,membership Three Month Digital Membership,% online purchases / total,% multiple seats purchase / total,% customers living in SF / total,quarter 2,quarter 3,quarter 4,weekday or weekend
count,846,846.000000,8.460000e+02,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000
mean,2022-02-08 23:34:28.085106432,359.223463,2.723682e+05,17367.539480,0.002364,0.026005,42.303783,0.319149,91.361473,54.679826,7.677936,0.212766,0.319149,0.260047,0.287234
min,2020-11-12 00:00:00,8.390000,7.039210e+01,5.000000,0.000000,0.000000,0.000000,0.000000,55.721393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2021-06-17 06:00:00,109.632500,1.201980e+04,1981.250000,0.000000,0.000000,9.000000,0.000000,85.956072,18.750000,1.219512,0.000000,0.000000,0.000000,0.000000
50%,2022-01-14 12:00:00,250.765000,6.288310e+04,14410.590000,0.000000,0.000000,22.000000,0.000000,92.920821,68.512310,4.804255,0.000000,0.000000,0.000000,0.000000
75%,2022-08-13 18:00:00,451.425000,2.037850e+05,22353.440000,0.000000,0.000000,44.000000,0.000000,100.000000,79.971264,10.960181,0.000000,1.000000,1.000000,1.000000
max,2023-10-16 00:00:00,2425.190000,5.881547e+06,768739.500000,1.000000,2.000000,1126.000000,10.000000,100.000000,100.000000,71.428571,1.000000,1.000000,1.000000,1.000000
std,NaN,378.809116,6.304185e+05,38168.195759,0.048593,0.166509,86.170005,0.910055,9.109505,32.583489,9.021352,0.409506,0.466423,0.438920,0.452739


In [11]:
X.corr()

,ad spent,membership Monthly Digital,% online purchases / total,% multiple seats purchase / total,% customers living in SF / total,quarter 2,quarter 3,quarter 4,weekday or weekend
ad spent,1.000000,-0.033406,-0.170879,0.315852,0.039295,-0.138433,0.116671,0.038301,-0.025820
membership Monthly Digital,-0.033406,1.000000,0.121329,-0.231606,0.103270,0.075101,-0.062630,-0.002185,-0.020774
% online purchases / total,-0.170879,0.121329,1.000000,-0.491845,-0.083942,0.097821,-0.155206,0.011152,-0.040915
% multiple seats purchase / total,0.315852,-0.231606,-0.491845,1.000000,0.072567,-0.233223,0.318973,0.106542,-0.038307
% customers living in SF / total,0.039295,0.103270,-0.083942,0.072567,1.000000,-0.055032,0.014677,0.077043,-0.027792
quarter 2,-0.138433,0.075101,0.097821,-0.233223,-0.055032,1.000000,-0.355934,-0.308193,0.001901
quarter 3,0.116671,-0.062630,-0.155206,0.318973,0.014677,-0.355934,1.000000,-0.405877,-0.003100
quarter 4,0.038301,-0.002185,0.011152,0.106542,0.077043,-0.308193,-0.405877,1.000000,-0.001140
weekday or weekend,-0.025820,-0.020774,-0.040915,-0.038307,-0.027792,0.001901,-0.003100,-0.001140,1.000000


In [12]:
import statsmodels.api as sm

# split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

# scale the data
binary_cols = ['quarter 2', 'quarter 3', 'quarter 4', 'weekday or weekend']
cols_to_scale = list(set(X.columns) - set(binary_cols))
X_scaled = X.copy()
# X_test_scaled = X_test.copy()
scaler = StandardScaler()
X_scaled[cols_to_scale] = scaler.fit_transform(X_scaled[cols_to_scale])
# X_test_scaled[cols_to_scale] = scaler.transform(X_test_scaled[cols_to_scale])

# OLS
X_scaled_with_intercept = sm.add_constant(X_scaled)
ols_model = sm.OLS(y, X_scaled_with_intercept).fit()

ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     live shows ticket sales   R-squared:                       0.134
Model:                                 OLS   Adj. R-squared:                  0.124
Method:                      Least Squares   F-statistic:                     14.35
Date:                     Tue, 05 Dec 2023   Prob (F-statistic):           1.03e-21
Time:                             15:03:45   Log-Likelihood:                -10064.
No. Observations:                      846   AIC:                         2.015e+04
Df Residuals:                          836   BIC:                         2.020e+04
Df Model:                                9                                         
Covariance Type:                 nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                              1.803e+04   2879.201      6.263      0.000    1.24e+04    2.37e+04
ad spent                            495.1416   1299.508      0.381      0.703   -2055.540    3045.823
membership Monthly Digital         7534.6387   1276.067      5.905      0.000    5029.967       1e+04
% online purchases / total         1341.8130   1422.482      0.943      0.346   -1450.243    4133.869
% multiple seats purchase / total  1.375e+04   1616.433      8.509      0.000    1.06e+04    1.69e+04
% customers living in SF / total   -213.4433   1246.934     -0.171      0.864   -2660.933    2234.046
quarter 2                          1067.9620   3802.826      0.281      0.779   -6396.246    8532.170
quarter 3                          1403.4072   3697.776      0.380      0.704   -5854.608    8661.423
quarter 4                         -1218.7627   3747.141     -0.325      0.745   -8573.672    6136.147
weekday or weekend                -3557.2206   2725.083     -1.305      0.192   -8906.028    1791.587
==============================================================================
Omnibus:                     1742.285   Durbin-Watson:                   1.833
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3151612.822
Skew:                          15.994   Prob(JB):                         0.00
Kurtosis:                     300.295   Cond. No.                         6.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
vif = pd.DataFrame()
vif["Features"] = X_scaled_with_intercept.columns
vif["VIF"] = [variance_inflation_factor(X_scaled_with_intercept, i) for i in range(X_scaled_with_intercept.shape[1])]

print(vif)

                            Features       VIF
0                              const  5.498357
1                           ad spent  1.120074
2         membership Monthly Digital  1.080030
3         % online purchases / total  1.342093
4  % multiple seats purchase / total  1.733023
5   % customers living in SF / total  1.031279
6                          quarter 2  1.606601
7                          quarter 3  1.970678
8                          quarter 4  1.792031
9                 weekday or weekend  1.008396


In [14]:
X = df.drop(['date', 'live shows ticket sales', 'ad spent',
            'membership Corporate Annual Digital', 'membership Corporate One Month Digital', 
            'membership Three Month Digital Membership'], axis=1)

y = df['live shows ticket sales']

# scale the data
binary_cols = ['quarter 2', 'quarter 3', 'quarter 4', 'weekday or weekend']
cols_to_scale = list(set(X.columns) - set(binary_cols))
X_scaled = X.copy()
# X_test_scaled = X_test.copy()
scaler = StandardScaler()
X_scaled[cols_to_scale] = scaler.fit_transform(X_scaled[cols_to_scale])
# X_test_scaled[cols_to_scale] = scaler.transform(X_test_scaled[cols_to_scale])

# OLS
X_scaled_with_intercept = sm.add_constant(X_scaled)
ols_model = sm.OLS(y, X_scaled_with_intercept).fit()

ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     live shows ticket sales   R-squared:                       0.134
Model:                                 OLS   Adj. R-squared:                  0.124
Method:                      Least Squares   F-statistic:                     14.33
Date:                     Tue, 05 Dec 2023   Prob (F-statistic):           1.10e-21
Time:                             15:04:48   Log-Likelihood:                -10064.
No. Observations:                      846   AIC:                         2.015e+04
Df Residuals:                          836   BIC:                         2.020e+04
Df Model:                                9                                         
Covariance Type:                 nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                              1.807e+04   2892.152      6.248      0.000    1.24e+04    2.37e+04
ad spent^2                           25.3495   1263.936      0.020      0.984   -2455.511    2506.210
membership Monthly Digital         7555.2628   1275.846      5.922      0.000    5051.026    1.01e+04
% online purchases / total         1331.2584   1422.436      0.936      0.350   -1460.707    4123.224
% multiple seats purchase / total   1.39e+04   1590.157      8.741      0.000    1.08e+04     1.7e+04
% customers living in SF / total   -209.2765   1246.995     -0.168      0.867   -2656.886    2238.333
quarter 2                           973.8253   3829.187      0.254      0.799   -6542.124    8489.774
quarter 3                          1383.8978   3714.463      0.373      0.710   -5906.870    8674.666
quarter 4                         -1249.5071   3753.834     -0.333      0.739   -8617.554    6118.540
weekday or weekend                -3571.7077   2725.179     -1.311      0.190   -8920.705    1777.289
==============================================================================
Omnibus:                     1741.623   Durbin-Watson:                   1.832
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3144412.250
Skew:                          15.981   Prob(JB):                         0.00
Kurtosis:                     299.954   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""